# Pods controllers

Deployment is the most popular way to run an application. This page discusses the components of deployment and common management patterns.

## ReplicaSet

A set of pods is deployed in parallel. K8s aims to maintain the specified number of replicas as specified in the replica set configuration.

---

The following cell creates a replica set.

In [2]:
kubectl create -f - << EOF
apiVersion: apps/v1
kind: ReplicaSet
metadata:
  name: frontend
spec:
  replicas: 3
  selector:
    matchLabels:
      tier: frontend
  template:
    metadata:
      labels:
        tier: frontend
    spec:
      containers:
      - name: php-redis
        image: us-docker.pkg.dev/google-samples/containers/gke/gb-frontend:v5
EOF

replicaset.apps/frontend created


The result is 3 pods (as specified in the `spec.replicas`). They all belong to the `frontent` replica set, that is clear from their naming.

In [ ]:
kubectl get pods

NAME             READY   STATUS    RESTARTS   AGE
frontend-4rpzj   1/1     Running   0          7s
frontend-6bnqz   1/1     Running   0          7s
frontend-b7v94   1/1     Running   0          7s


You can use the command `kubectl get rs` to list replica sets.

In [ ]:
kubectl get rs

NAME       DESIRED   CURRENT   READY   AGE
frontend   3         3         3       8s


Use the command `kubectl delete rs frontend` to delete the replica set.

In [3]:
kubectl delete rs frontend

replicaset.apps "frontend" deleted from default namespace


## Deployment

Deployment is the most popular way to run an application. It runs and manages a **replica set**, which is a set of pods that run in parallel. A deployment is generally tied to a single ReplicaSet, but it implements a **rolling update** mechanism. If the ReplicaSet's configuration must be changed, the deployment retains the ReplicaSet's old configration until it ensures that the newly deployed updates are healthy.

---

The following cell uses an imperative approach to create a deployment.

In [ ]:
kubectl create deployment nginx-deployment --image=nginx:1.22.1 --replicas=3

deployment.apps/nginx-deployment created


You can view a list of available deployments by entering the command `kubectl get deployment`.

In [ ]:
kubectl get deployment 

NAME               READY   UP-TO-DATE   AVAILABLE   AGE
nginx-deployment   3/3     3            3           102s


As was specified by the `replicas=3`, three identical pods have been deployed.

In [ ]:
kubectl get pods

NAME                                READY   STATUS    RESTARTS   AGE
nginx-deployment-799db84976-fcbjb   1/1     Running   0          112s
nginx-deployment-799db84976-fwmng   1/1     Running   0          112s
nginx-deployment-799db84976-hs492   1/1     Running   0          112s


In [ ]:
kubectl delete deployment nginx-deployment

deployment.apps "nginx-deployment" deleted from default namespace


## DaemonSet

The DaemonSet is a controller that runs a single instance of the same pod on each node in the cluster.

---

The following cell runs the daemon set on the new cluster consisting of three nodes.

In [3]:
minikube start --profile daemonset-example --nodes 3 &> /dev/null
kubectl create -f - << EOF
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: fluentd-agent
  namespace: default
  labels:
    k8s-app: fluentd-agent
spec:
  selector:
    matchLabels:
      k8s-app: fluentd-agent
  template:
    metadata:
      labels:
        k8s-app: fluentd-agent
    spec:
      containers:
      - name: fluentd
        image: quay.io/fluentd_elasticsearch/fluentd:v4.5.2
EOF

daemonset.apps/fluentd-agent created


The following cell shows the pods in the cluster.

In [4]:
kubectl get pods -o wide

NAME                  READY   STATUS              RESTARTS   AGE   IP       NODE                    NOMINATED NODE   READINESS GATES
fluentd-agent-f7hl4   0/1     ContainerCreating   0          0s    <none>   daemonset-example-m02   <none>           <none>
fluentd-agent-wqkhj   0/1     ContainerCreating   0          0s    <none>   daemonset-example       <none>           <none>
fluentd-agent-ztwl4   0/1     ContainerCreating   0          0s    <none>   daemonset-example-m03   <none>           <none>


The `NODE` column of the output is important here - each pod is on uniqe node, and the DaemonSet garantees that.

In [7]:
minikube delete --profile daemonset-example &> /dev/null